# Initialization

Test notebook for the DataHandlers. Test the CMAPSS DataHandler

First we import the necessary packages and create the global variables.

In [1]:
import numpy as np
import math
import tensorflow as tf

from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D, LSTM, CuDNNLSTM
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

import CMAPSAuxFunctions

from numpy.random import seed
from tensorflow import set_random_seed


Using TensorFlow backend.


# Define different types of Keras models

In [2]:
l2_lambda_regularization = 0.2
#l1_lambda_regularization = 0.2

def RULmodel_LSTM(input_shape):
    """Define the RNN model"""
    
    #Create a sequential model
    model = Sequential()
    #model.add(Masking(mask_value=0, imput))
    #model.add(LSTM(input_shape=input_shape, units=20, return_sequences=False, dropout=0.2, name='lstm1'))
    model.add(CuDNNLSTM(input_shape=input_shape, units=20, return_sequences=False, name='lstm1'))
    model.add(Dropout(0.2))
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='out'))
    
    return model

# Create the Data Handler

In [3]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
#selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])

selected_indices = np.array(list(range(1,22)))

selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 0

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

# Create Tunable Model

In [4]:
#min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
#scaler = StandardScaler()
scaler = MinMaxScaler()

In [5]:
def get_compiled_model(shape, model_type='ann'):
    
    K.clear_session()  #Clear the previous tensorflow graph
    
    #To test the model without randomness

    
    #Shared parameters for the models
    optimizer = Adam(lr=0.001,beta_1=0.5)
    
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models

    model = RULmodel_LSTM(shape)
    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    return model

In [6]:
num_features = len(selected_features)

shape = (window_size, num_features)
model = get_compiled_model(shape, model_type='lstm')
tModel = SequenceTunableModelRegression('ModelRUL_LSTM_1', model, lib_type='keras', data_handler=dHandler_cmaps)

# Load Data

In [7]:
tModel.data_handler.sequence_length = 30
#tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
tModel.data_handler.sequence_stride = 1
tModel.data_handler.max_rul = 0

In [7]:
# #Load sequenced data (do not unroll sequence into a single feature vector)

tModel.data_handler.data_scaler = scaler
tModel.data_scaler = None

tModel.load_data(unroll=False, verbose=1, cross_validation_ratio=0.01)
tModel.print_data()

Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 125. Cros-Validation ratio 0.01
Loading data from file and computing dataframes
cv number
[55]
Printing shapes

Training data (X, y)
(17567, 30, 14)
(17567, 1)
Cross-Validation data (X, y)
(1, 30, 14)
(1, 1)
Testing data (X, y)
(100, 30, 21)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[[0.         0.18373494 0.40680183 ... 0.         0.71317829 0.7246617 ]
  [0.         0.28313253 0.4530194  ... 0.         0.66666667 0.73101353]
  [0.         0.34337349 0.36952256 ... 0.         0.62790698 0.62137531]
  ...
  [0.         0.34337349 0.25724875 ... 0.         0.6744186  0.53838719]
  [0.         0.21084337 0.30063222 ... 0.         0.6124031  0.64277824]
  [0.         0.29819277 0.49008066 ... 0.         0.70542636 0.71361502]]

 [[0.         0.28313253 0.4530194  ... 0.         0.66666667 0.73101353]
  [0.         0.34337349 0.36952256 ... 0.         0.62790698 0.62137531]
  [0.         0.34337349

/home/controlslab/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/controlslab/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/media/controlslab/DATA/Projects/NASA_RUL_-CMAPS-/code/data_handler_CMAPS.py:133: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  norm_test_df = pd.DataFrame(self._data_scaler.transform(self._df_test[cols_normalize]), columns=cols_normalize, index=self._df_test.index)


# Train the model and test some Tunable Model functionalities

In [ ]:
nFeatures = len(selected_features)

lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

shape = (tModel.data_handler.sequence_length, nFeatures)
model = get_compiled_model(shape, model_type='lstm')
tModel.change_model('ModelRUL_RNN_1', model, 'keras')

tModel.epochs = 100

tModel.train_model(learningRate_scheduler=lrate, verbose=1)

Epoch 1/100
17731/17731 [==============================] - 3s 159us/step - loss: 8124.7990 - mean_squared_error: 8120.3017
Epoch 2/100
17731/17731 [==============================] - 0s 12us/step - loss: 7349.1129 - mean_squared_error: 7344.1392
Epoch 3/100
17731/17731 [==============================] - 0s 12us/step - loss: 6273.7442 - mean_squared_error: 6267.9084
Epoch 4/100
17731/17731 [==============================] - 0s 12us/step - loss: 5267.1569 - mean_squared_error: 5260.1987
Epoch 5/100
17731/17731 [==============================] - 0s 12us/step - loss: 4307.9284 - mean_squared_error: 4299.6841
Epoch 6/100
17731/17731 [==============================] - 0s 12us/step - loss: 3421.1964 - mean_squared_error: 3411.5036
Epoch 7/100
17731/17731 [==============================] - 0s 12us/step - loss: 2634.1178 - mean_squared_error: 2622.8905
Epoch 8/100
17731/17731 [==============================] - 0s 11us/step - loss: 1996.1576 - mean_squared_error: 1983.3752
Epoch 9/100
17731/17731

In [ ]:
tModel.evaluate_model(['rhs', 'rmse'], round=2)
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_test)+1)

#print(tModel.y_predicted)
#print(tModel.y_predicted_rounded)
#print(tModel.y_test)

for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, np.ravel(tModel.y_predicted), tModel.y_predicted_rounded, np.ravel(tModel.y_test)):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

In [9]:
for start, stop in zip(range(0, 112), range(50, 192)):
    print(start, stop)

0 50
1 51
2 52
3 53
4 54
5 55
6 56
7 57
8 58
9 59
10 60
11 61
12 62
13 63
14 64
15 65
16 66
17 67
18 68
19 69
20 70
21 71
22 72
23 73
24 74
25 75
26 76
27 77
28 78
29 79
30 80
31 81
32 82
33 83
34 84
35 85
36 86
37 87
38 88
39 89
40 90
41 91
42 92
43 93
44 94
45 95
46 96
47 97
48 98
49 99
50 100
51 101
52 102
53 103
54 104
55 105
56 106
57 107
58 108
59 109
60 110
61 111
62 112
63 113
64 114
65 115
66 116
67 117
68 118
69 119
70 120
71 121
72 122
73 123
74 124
75 125
76 126
77 127
78 128
79 129
80 130
81 131
82 132
83 133
84 134
85 135
86 136
87 137
88 138
89 139
90 140
91 141
92 142
93 143
94 144
95 145
96 146
97 147
98 148
99 149
100 150
101 151
102 152
103 153
104 154
105 155
106 156
107 157
108 158
109 159
110 160
111 161
